In [11]:
import pandas as pd
import numpy as np
import os
import re
import string
import warnings

warnings.filterwarnings('ignore')

path = os.getcwd()

In [2]:
def html_tag_remover(text):
    re_html = re.compile('<.*?>')
    new_text = re.sub(re_html, '', text)
    new_text = remove_url(new_text)
    return new_text

def remove_url(text):
    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", text).split())


In [3]:
df = pd.read_csv(os.path.join(path, 'resources', 'newdf.csv'))
df.head()

,Unnamed: 0,JobID,WindowID,Title,Description,Requirements,City,State,Country,Zip5,StartDate,EndDate
0,0,1,1,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,Washington,DC,US,20531.0,17:01.6,4/6/2012 23:59
1,1,4,1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,Charlotte,NC,US,28217.0,03:44.1,4/20/2012 23:59
2,2,7,1,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,Winter Park,FL,US,32792.0,36:55.4,4/1/2012 23:59
3,3,8,1,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,01:10.1,4/2/2012 23:59
4,4,9,1,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,Orlando,FL,US,NaN,01:11.9,4/2/2012 23:59


### Data Preprocessing
#### Menghilangkan Beberapa Kolom TIdak Berguna

In [4]:
df_copy = df.copy()

useless_col = ['WindowID', 'JobID', 'City', 'State', 'StartDate', 'EndDate', 'Zip5', 'Unnamed: 0']
df_copy = df_copy.drop(useless_col, axis = 1)
df_copy.head()

,Title,Description,Requirements,Country
0,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,US
1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,US
2,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,US
3,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,US
4,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,US


In [5]:
df_copy.Country.value_counts()

US    1000
Name: Country, dtype: int64

In [6]:
df_copy.Requirements.isna().value_counts()

False    997
True       3
Name: Requirements, dtype: int64

In [7]:
df_copy = df_copy.dropna()
df_copy = df_copy.drop_duplicates()

In [8]:
df_copy['Country'] = df_copy.Country.apply(lambda x : 'ID')
df_copy.head()

,Title,Description,Requirements,Country
0,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,ID
1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,ID
2,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,ID
3,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,ID
4,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,ID


In [12]:
# df_copy['Description'] = df_copy.Description.apply(lambda x : html_tag_remover(x))
# df_copy['Requirements'] = df_copy.Requirements.apply(lambda x : html_tag_remover(x))

removePattern = r'(<(.*?)>)|(&\w+)'
addSpacePattern = r'([;:])|(\\r)|(\\n)'
removeExtraSpaces = r'(\s\s+?)(?=\S)'
df_copy['DescCleaned'] = df_copy['Description'].str.lower()
df_copy['DescCleaned'] = df_copy['DescCleaned'].str.replace(removePattern, "")
df_copy['DescCleaned'] = df_copy['DescCleaned'].str.replace(addSpacePattern, " ")
df_copy['DescCleaned'] = df_copy['DescCleaned'].str.replace(removeExtraSpaces, " ")
# Same for Requirements
df_copy['ReqCleaned'] = df_copy['Requirements'].str.lower()
df_copy['ReqCleaned'] = df_copy['ReqCleaned'].str.replace(removePattern, "")
df_copy['ReqCleaned'] = df_copy['ReqCleaned'].str.replace(addSpacePattern, " ")
df_copy['ReqCleaned'] = df_copy['ReqCleaned'].str.replace(removeExtraSpaces, " ")

df_copy.head()

,Title,Description,Requirements,Country,DescCleaned,ReqCleaned
0,Security Engineer/Technical Lead,<p>Security Clearance Required:&nbsp; Top Secr...,<p>SKILL SET</p>\r<p>&nbsp;</p>\r<p>Network Se...,ID,security clearance required top secret job num...,skill set network security tools webdefend web...
1,SAP Business Analyst / WM,<strong>NO Corp. to Corp resumes&nbsp;are bein...,<p><b>WHAT YOU NEED: </b></p>\r<p>Four year co...,ID,no corp. to corp resumes are being considered ...,what you need four year college degree minimum...
2,P/T HUMAN RESOURCES ASSISTANT,<b> <b> P/T HUMAN RESOURCES ASSISTANT</b> <...,Please refer to the Job Description to view th...,ID,p/t human resources assistant —— 1-2 years ex...,please refer to the job description to view th...
3,Route Delivery Drivers,CITY BEVERAGES Come to work for the best in th...,Please refer to the Job Description to view th...,ID,city beverages come to work for the best in th...,please refer to the job description to view th...
4,Housekeeping,I make sure every part of their day is magica...,Please refer to the Job Description to view th...,ID,i make sure every part of their day is magical...,please refer to the job description to view th...


In [14]:
df_copy.loc[0].DescCleaned

'security clearance required top secret job number tmr-447 location of job washington, dc tmr, inc. is an equal employment opportunity company for more job opportunities with tmr, visit our website www.tmrhq.com send resumes to hr@tmrhq2.com job summary leads the customer s overall cyber security strategy, formalizes service offerings consisted with itil best practices, and provides design and architecture support. provide security design / architecture support for ojp s it security division (itsd) leads the secops team in the day to day ojp security operations support provides direction when needed in a security incident or technical issues works in concert with network operations on design /integration for best security posture supports business development functions including capture management, proposal development and responses, and other initiatives to include conferences, trade shows, webinars, developing white papers and the like. identifies resources and mentors in-house talen